# TODO
* The most exciting things that could improve the model
* Unfortunatelly, I spent 10 days working on it, and the exam period is slowly coming :D

### HYPERARAMETERS OPTUNA
* WIP
* Documentation is perfect
* https://optuna.org/#code_examples

In [ ]:
def objective(space):
    """
    study minimizes this function 
    return: evaluation metric which we minimize
    """
    param={
        'silent': 1,
        'objective': 'binary:logistic',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)}
        
        #'max_depth': hp.quniform("max_depth", 3, 18, 1),
        #'gamma': hp.uniform ('gamma', 1,9),
        #'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        #'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        #'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        #'learning_rate'    : hp.uniform('learning_rate', 0.0005, 0.3),
        #'early_stopping_rounds': hp.uniform ('early_stopping_rounds', 10,600)}
    reg=xgb.XGBRegressor(
                    max_depth = int(space['max_depth']),
                    gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'],
                    reg_lambda = space['reg_lambda'],
                    colsample_bytree=space['colsample_bytree'],
                    min_child_weight = space['min_child_weight'],
                    learning_rate=space['learning_rate'],
                    early_stopping_rounds=space['early_stopping_rounds'])   
    reg.fit(X_train, y_train,
            eval_set=[(X_train,y_train),(X_test,y_test)],
            verbose=False)
    pred = reg.predict(X_test)
    accuracy = mean_absolute_error(test['Target_Volume'],pred)
    print ("SCORE: "+ str(accuracy))
    return {'loss': accuracy, 'status': STATUS_OK }

In [ ]:


trials = Trials()

best_hyperparams = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=50,trials=trials)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)

### K-FOLD CROSS VALDIATION
* Use the best model and run it using k-fold cross-validation
* TODO: Check which results are better

In [ ]:
from sklearn.model_selection import TimeSeriesSplit #k-fold crossvalidation

tss = TimeSeriesSplit(n_splits=10, test_size=252) #half year
fig, axs = plt.subplots(10, 1, figsize=(15, 15), sharex=True,constrained_layout = True)

fold = 0
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    train[TARGET].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test[TARGET].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1

In [ ]:
df = df.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    train = add_dates(train)
    test = add_dates(test)

    FEATURES = ['Close', 'ATR', 'RSI', 'Dayofweek', 'Dayofmonth', 'Month']
    TARGET = 'Volume'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

In [ ]:
#print(f'Score across folds {np.mean(scores):0.4f}')
#print(f'Fold scores:{scores}')

# PERSONAL WORKSPACE

## NORMALIZE
* Faster Convergence
* Correct axis ratio if we want for example compute distances between values
* FastRun argument: Takes around 40s to generate, generate only if necessary
* IMPORTANT: Normalize it on training data, otherwise we use future data
* XGBoost does not need normalizing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

if Normalization:
    sc = MinMaxScaler()
    sc_train = sc.fit_transform(train)
    sc_test = sc.transform(test)
    train=pd.DataFrame(sc_train,columns=train.columns,index=train.index)
    test=pd.DataFrame(sc_test,columns=test.columns,index=test.index)
    test.tail()

### NORMALIZED DISTRIBUTION
* Check if the data are evenly distributed after normalization
* FastRun argument: Takes around 40s to generate, generate only if necessary

In [ ]:
if Normalization:
    cols = df.columns.tolist()
    fig, axs = plt.subplots(len(cols), 1,figsize=(15, len(cols)*3)) #TODO: zobrazit vsechno rozdeleni se spolecnou x osou, at zjistime, zda neco neni skew
    plt.suptitle('Normalised Distribution')
    for i in range(len(cols)):
        df[cols[i]].plot(kind='hist',ax=axs[i],title=cols[i])
    plt.savefig('Distribution Normalised.pdf')

### Bonus: Saving Model For later

In [ ]:
# Save model
reg.save_model('model.json')

In [ ]:
reg_new = xgb.XGBRegressor()
reg_new.load_model('model.json')
future_w_features['pred'] = reg_new.predict(future_w_features[FEATURES])
future_w_features['pred'].plot(figsize=(10, 5),
                               ms=1, lw=1,
                               title='Future Predictions')

## PROPHET
- Prophet model expects the dataset to be named a specific way. We will rename our dataframe columns before feeding it into the model.
    - Datetime column named: `ds`
    - target : `y`

In [ ]:
# Format data for prophet model using ds and y
prophet_train = pjme_train.reset_index() \
    .rename(columns={'Datetime':'ds',
                     'PJME_MW':'y'})

In [ ]:
%%time
model = Prophet()
model.fit(pjme_train_prophet)

In [ ]:
# Predict on test set with model
pjme_test_prophet = pjme_test.reset_index() \
    .rename(columns={'Datetime':'ds',
                     'PJME_MW':'y'})

pjme_test_fcst = model.predict(pjme_test_prophet)

In [ ]:
pjme_test_fcst.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
fig = model.plot(pjme_test_fcst, ax=ax)
ax.set_title('Prophet Forecast')

In [ ]:
fig = model.plot_components(pjme_test_fcst)
 

### Compare Forecast to Actuals

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(figsize=(15, 5))
ax.scatter(pjme_test.index, pjme_test['PJME_MW'], color='r')
fig = model.plot(pjme_test_fcst, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(pjme_test.index, pjme_test['PJME_MW'], color='r')
fig = model.plot(pjme_test_fcst, ax=ax)
ax.set_xbound(lower='01-01-2015',
              upper='02-01-2015')
ax.set_ylim(0, 60000)
plot = plt.suptitle('January 2015 Forecast vs Actuals')

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(figsize=(15, 5))
ax.scatter(pjme_test.index, pjme_test['PJME_MW'], color='r')
fig = model.plot(pjme_test_fcst, ax=ax)
ax.set_xbound(lower='01-01-2015', upper='01-08-2015')
ax.set_ylim(0, 60000)
ax.set_title('First Week of January Forecast vs Actuals')
 

### Evaluate the model with Error Metrics

In [ ]:
np.sqrt(mean_squared_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst['yhat']))

In [ ]:
mean_absolute_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst['yhat'])

In [ ]:
mean_absolute_percentage_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst['yhat'])

### Adding Holidays

Next we will see if adding holiday indicators will help the accuracy of the model. Prophet comes with a Holiday Effects parameter that can be provided to the model prior to training.

We will use the built in pandas USFederalHolidayCalendar to pull the list of holidays

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()


holidays = cal.holidays(start=pjme.index.min(),
                        end=pjme.index.max(),
                        return_name=True)
holiday_df = pd.DataFrame(data=holidays,
                          columns=['holiday'])
holiday_df = holiday_df.reset_index().rename(columns={'index':'ds'})

In [ ]:
%%time
model_with_holidays = Prophet(holidays=holiday_df)
model_with_holidays.fit(pjme_train_prophet)

In [ ]:
# Predict on training set with model
pjme_test_fcst_with_hols = \
    model_with_holidays.predict(df=pjme_test_prophet)

In [ ]:
fig = model_with_holidays.plot_components(
    pjme_test_fcst_with_hols)
 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(pjme_test.index, pjme_test['PJME_MW'], color='r')
fig = model.plot(pjme_test_fcst_with_hols, ax=ax)
ax.set_xbound(lower='07-01-2015',
              upper='07-07-2015')
ax.set_ylim(0, 60000)
plot = plt.suptitle('July 4 Predictions vs Actual')

In [ ]:
np.sqrt(mean_squared_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst_with_hols['yhat']))

In [ ]:
mean_absolute_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst_with_hols['yhat'])

In [ ]:
mean_absolute_percentage_error(y_true=pjme_test['PJME_MW'],
                   y_pred=pjme_test_fcst_with_hols['yhat'])

### Predict into the Future

We can use the built in `make_future_dataframe` method to build our future dataframe and make predictions.

In [ ]:
future = model.make_future_dataframe(periods=365*24, freq='h', include_history=False)
forecast = model_with_holidays.predict(future)

In [ ]:
forecast[['ds','yhat']].head()

### Predicting the Future
- Retraining on all data
- To Predict the future we need an emtpy dataframe for future date ranges.
- Run those dates through our feature creation code + lag creation

In [ ]:
# Retrain on all data
df = add_dates(df)

FEATURES = ['Close', 'ATR', 'RSI', 'Dayofweek', 'Dayofmonth', 'Month']
TARGET = 'Volume'

X_all = df[FEATURES]
y_all = df[TARGET]

reg = xgb.XGBRegressor(base_score=0.5,
                       booster='gbtree',    
                       n_estimators=500,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.01)
reg.fit(X_all, y_all,
        eval_set=[(X_all, y_all)],
        verbose=100)

In [ ]:
df.index.max()

In [ ]:
# Create future dataframe
future = pd.date_range('2018-08-03','2019-08-01', freq='1d')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_future = pd.concat([df, future_df])
df_and_future = add_dates(df_and_future)
df_and_future = add_lags(df_and_future)

In [ ]:
future_w_features = df_and_future.query('isFuture').copy()

### Predict the future

In [ ]:
future_w_features['pred'] = reg.predict(future_w_features[FEATURES])

In [ ]:
future_w_features['pred'].plot(figsize=(10, 5),
                               ms=1,
                               lw=1,
                               title='Future Predictions')
 

## Model Building

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import classification_report

#### Rozdělení na trénovací a testovací data

In [ ]:
x = data[['age', 'hypertension', 'heart_disease','bmi', 'gender_encoded', 'married_encoded','worktype_encoded','residence_encoded','smoker_encoded']]
y = data['stroke']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
sc = StandardScaler() # standardization of test and train set
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
print("Shape of x_train:{}".format(x_train.shape))
print("Shape of x_test:{}".format(x_test.shape))
print("Shape of y_train:{}".format(y_train.shape))
print("Shape of y_test:{}".format(y_test.shape))

#### Baseline
* the most basic algorithm, where we find whether is more probable to have or not have stroke in trainign dataset
* in the testing dataset, that everyone has or does not have a troke based on the first point
* useful to check whether other algorithms are working
* in this case the moct common scenario is NO STROKE

In [ ]:
# BASELINE
y_prediction = np.zeros(len(x_test), dtype=int)
accuracy = accuracy_score(y_test, y_prediction)

#### KNN
* implemented for 1-10 nearost neigbors
* runs the algorithm 9 times in total and use the k-negbors with the best result

In [ ]:
# KNN
best_accuracy = 0
best_k = 0
for k in range (1,10):
    model1 = KNeighborsClassifier(k).fit(x_train, y_train) # serach for k closest neigbors
    y_prediction1 = model1.predict(x_test)  # predict response
    accuracy1 = max(best_accuracy, accuracy_score(y_test, y_prediction1))
    if accuracy1 > best_accuracy: best_k = k
    best_accuracy = accuracy1

 #### DECISION TREE

In [ ]:
# DECISION TREE
decisiontree = DecisionTreeClassifier()  # Create models
model2 = decisiontree.fit(x_train, y_train)
y_prediction2 = model2.predict(x_test)  # predict response
accuracy2 = accuracy_score(y_test, y_prediction2)

#### SVM

In [ ]:
# SVM
svmmodel = svm.SVC(kernel='linear', probability=True)  # Linear Kernel
model3 = svmmodel.fit(x_train, y_train)
y_prediction3 = model3.predict(x_test)  # predict response
accuracy3 = accuracy_score(y_test, y_prediction3)

#### LOGISTIC REGRESSION

In [ ]:
# LOGISTIC REGRESSION
logisticregression = LogisticRegression()
model4 = logisticregression.fit(x_train, y_train)
y_prediction4 = model4.predict(x_test)  # predict response
accuracy4 = accuracy_score(y_test, y_prediction4)

## Results
* Baseline: dummiest, other algorithms should be able to be more accure
* KNN, Decision Tree, SVM, Logistic Regression

Decision trees are working the worst in this scenario, the rest is giving as the same results as baseline, which shows how hard the task is. As the majority of results are "no stroke", the algorithm learns on his own that it is the best to say that everything is not a stroke.

#### Predictions table

In [ ]:
report = pd.DataFrame()
report['Actual values'] = y_test
report['Predicted values KNN'] = y_prediction1
report['Predicted values Decision tree'] = y_prediction2
report['Predicted values SVM'] = y_prediction3
report['Predicted values Logistic Regression'] = y_prediction4
report.to_csv('predictions.csv') #save our predictions into csv file

#### Accuracies

In [ ]:
print("Baseline: ",accuracy)
print("KNN: ",accuracy1, "(for "+str(best_k)+" closest neigbors)")
print("Decision Trees: ", accuracy2)
print("SVM :", accuracy3)
print("Logistic Regression: ", accuracy4)

#### CONFUSION MATRIX

In [ ]:
ConfusionMatrix1 = confusion_matrix(y_test, y_prediction1)
ConfusionMatrix2 = confusion_matrix(y_test, y_prediction2)
ConfusionMatrix3 = confusion_matrix(y_test, y_prediction3)
ConfusionMatrix4 = confusion_matrix(y_test, y_prediction4)

# KNN
plt.subplots(2, 2, figsize=(10, 7))
plt.suptitle('Confusion Matrix')
plt.subplot(2, 2, 1)
ax = sns.heatmap(ConfusionMatrix1, annot=True, cmap="YlGnBu")
plt.title("KNN")
ax.set_ylabel('Actual values')

# DECISION TREE
plt.subplot(2, 2, 2)
ax = sns.heatmap(ConfusionMatrix2, annot=True, cmap="YlGnBu")
plt.title("Decision Tree")
ax.set_ylabel('Actual values')

# SVM
plt.subplot(2, 2, 3)
ax = sns.heatmap(ConfusionMatrix3, annot=True, cmap="YlGnBu")
plt.title("SVM")
ax.set_xlabel('Predicted values')
ax.set_ylabel('Actual values')

# LOGISTIC REGRESSION
plt.subplot(2, 2, 4)
ax = sns.heatmap(ConfusionMatrix4, annot=True, cmap="YlGnBu")
plt.title("Logistic Regression")
ax.set_xlabel('Predicted values')
ax.set_ylabel('Actual values')
plt.savefig('Confusion Matrix.pdf')

#### ROS CURVE

In [ ]:
    plt.subplots(2, 2, figsize=(10, 7))
    plt.suptitle('ROC Curve')
    plt.subplot(2, 2, 1)
    train_probs1 = model1.predict_proba(x_train)
    train_probs1 = train_probs1[:, 1]
    fpr1_train, tpr1_train, _ = roc_curve(y_train, train_probs1)  # pos_label=1
    test_probs1 = model1.predict_proba(x_test)
    test_probs1 = test_probs1[:, 1]
    fpr1_test, tpr1_test, _ = roc_curve(y_test, test_probs1)  # pos_label=1
    plt.plot(fpr1_train, tpr1_train, marker='.', label='train')
    plt.plot(fpr1_test, tpr1_test, marker='.', label='validation')
    plt.title('KNN')
    plt.ylabel('True Positive Rate')
    plt.legend()

    # ROC CURVE DECISION TREE
    plt.subplot(2, 2, 2)
    train_probs2 = model2.predict_proba(x_train)
    train_probs2 = train_probs2[:, 1]
    fpr2_train, tpr2_train, _ = roc_curve(y_train, train_probs2)  # pos_label=1
    test_probs2 = model2.predict_proba(x_test)
    test_probs2 = test_probs2[:, 1]
    fpr2_test, tpr2_test, _ = roc_curve(y_test, test_probs2)  # pos_label=1
    plt.plot(fpr2_train, tpr2_train, marker='.', label='train')
    plt.plot(fpr2_test, tpr2_test, marker='.', label='validation')
    plt.title('Decision Tree')
    plt.ylabel('True Positive Rate')
    plt.legend()

    # ROC CURVE MODEL SVM
    plt.subplot(2, 2, 3)
    train_probs3 = model3.predict_proba(x_train)
    train_probs3 = train_probs3[:, 1]
    fpr3_train, tpr3_train, _ = roc_curve(y_train, train_probs3)  # pos_label=1
    test_probs3 = model3.predict_proba(x_test)
    test_probs3 = test_probs3[:, 1]
    fpr3_test, tpr3_test, _ = roc_curve(y_test, test_probs3)  # pos_label=1
    plt.plot(fpr3_train, tpr3_train, marker='.', label='train')
    plt.plot(fpr3_test, tpr3_test, marker='.', label='validation')
    plt.title('SVM')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()

    # ROC CURVE LOGISTIC REGRESSION
    plt.subplot(2, 2, 4)
    train_probs4 = model4.predict_proba(x_train)
    train_probs4 = train_probs4[:, 1]
    fpr4_train, tpr4_train, _ = roc_curve(y_train, train_probs4)  # pos_label=1
    test_probs4 = model4.predict_proba(x_test)
    test_probs4 = test_probs4[:, 1]
    fpr4_test, tpr4_test, _ = roc_curve(y_test, test_probs4)  # pos_label=1
    plt.plot(fpr4_train, tpr4_train, marker='.', label='train')
    plt.plot(fpr4_test, tpr4_test, marker='.', label='validation')
    plt.title('Logistic Regression')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.savefig('ROC Curve.pdf')